In [1]:
# Plotting suite update: ALWAYS highlight 0s for any TARGET variable
# Saves EVERY figure as a PDF in the current working directory.
# Targets' time series: red markers at zeros
# Targets' histograms: overlay a red "zero bar" showing count of zeros
# Targets' scatters (vs ENV features): red points where target==0
# Supports ENV_VARS stored as single columns of 24-hour FLOAT vectors (00..23)
# Computes FULL-day and OVERNIGHT (prev 18:00–23:59 + curr 00:00–05:59) stats

import ast
import re
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# config
CSV_PATH     = "Final_2015_Data_Site8.csv"  # <-- set this
DATE_COL     = "Date"
WINDOW_START = "2015-07-01"
WINDOW_END   = "2015-10-31"

PRECIP_COL   = "Precipitation"   # daily scalar
ENV_VARS     = ["Temperature", "Humidity", "Dew_Point", "Water_Temperature"]  # each row holds a 24h FLOAT vector (00..23) in ONE column
TARGETS      = ["Adults_8_Col", "Adults_8_Gam0"]  # zeros treated as missing (highlighted)

SAVE_AGGREGATED_CSV = False
AGG_OUT_PATH = "aggregated_env_features.csv"

# saving controls (all PDFs saved here; default is current dir)
SAVE_PDFS = True
SAVE_DIR = Path(".")  # current working directory
_SAVE_COUNT = {}

def _slug(s: str) -> str:
    return re.sub(r"[^A-Za-z0-9._-]+", "_", str(s)).strip("_")

def _savefig_pdf(name: str):
    """Save current matplotlib figure as a PDF with a clean, collision-safe filename."""
    if not SAVE_PDFS:
        return None
    SAVE_DIR.mkdir(parents=True, exist_ok=True)
    name = _slug(name)
    n = _SAVE_COUNT.get(name, 0)
    _SAVE_COUNT[name] = n + 1
    fname = f"{name}.pdf" if n == 0 else f"{name}__{n+1}.pdf"
    path = SAVE_DIR / fname
    plt.savefig(path, bbox_inches="tight")
    plt.close()  # free memory for large batches
    print(f"[saved] {path}")
    return str(path)

# HELPERS
def _standardize_columns(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.columns = [c.strip() for c in df.columns]
    return df

def _parse_and_sort_dates(df: pd.DataFrame, date_col: str) -> pd.DataFrame:
    if date_col not in df.columns:
        raise ValueError(f"DATE_COL '{date_col}' not found in CSV.")
    df = df.copy()
    df[date_col] = pd.to_datetime(df[date_col])
    df = df.sort_values(date_col).reset_index(drop=True)
    return df

def _filter_window(df: pd.DataFrame, date_col: str, start: str, end: str) -> pd.DataFrame:
    mask = (df[date_col] >= pd.to_datetime(start)) & (df[date_col] <= pd.to_datetime(end))
    return df.loc[mask].reset_index(drop=True)

def _parse_vec24_cell(cell) -> np.ndarray:
    """
    Parse a 24-length vector cell robustly into float array of length 24.
    Accepts:
      - Python/JSON-like list strings: "[1, 2, 3, ...]"
      - Comma-separated string: "1,2,3,..."
      - Whitespace-separated: "1 2 3 ..."
      - Already list/tuple/array
    Returns np.array of float(24), or np.full(24, np.nan) if parsing fails/length != 24.
    """
    if isinstance(cell, (list, tuple, np.ndarray)):
        arr = np.array(cell, dtype=float)
    else:
        s = str(cell).strip()
        arr = None
        try:
            parsed = ast.literal_eval(s)
            if isinstance(parsed, (list, tuple, np.ndarray)):
                arr = np.array(parsed, dtype=float)
        except Exception:
            arr = None
        if arr is None:
            s2 = s.replace("[", "").replace("]", "")
            parts = [p.strip() for p in (s2.split(",") if "," in s2 else s2.split())]
            try:
                arr = np.array([float(x) if x != "" else np.nan for x in parts], dtype=float)
            except Exception:
                return np.full(24, np.nan, dtype=float)
    if arr.size != 24:
        return np.full(24, np.nan, dtype=float)
    return arr.astype(float)

def _env_matrix_from_vector_column(df: pd.DataFrame, base: str) -> np.ndarray:
    if base not in df.columns:
        raise ValueError(f"ENV column '{base}' not found in CSV.")
    vecs = df[base].apply(_parse_vec24_cell).to_numpy()
    mat = np.vstack(vecs)  # (n, 24)
    if mat.shape[1] != 24:
        raise ValueError(f"ENV '{base}' did not parse to 24 values per row.")
    return mat

def _full_day_stats(mat24: np.ndarray) -> dict[str, np.ndarray]:
    return {
        "FULL_min":    np.nanmin(mat24, axis=1),
        "FULL_max":    np.nanmax(mat24, axis=1),
        "FULL_median": np.nanmedian(mat24, axis=1),
        "FULL_sd":     np.nanstd(mat24, axis=1, ddof=0),
    }

def _overnight_stats(mat24: np.ndarray) -> dict[str, np.ndarray]:
    """
    For date i: prev day hours 18..23 + current day 0..5  -> align to current date (i).
    First row has no previous day -> NaNs.
    """
    n = mat24.shape[0]
    ov = np.full((n, 12), np.nan, dtype=float)
    if n >= 2:
        prev_18_23 = mat24[:-1, 18:24]
        curr_00_05 = mat24[1:,  0:6]
        ov[1:, :] = np.concatenate([prev_18_23, curr_00_05], axis=1)
    return {
        "OV_min":    np.nanmin(ov, axis=1),
        "OV_max":    np.nanmax(ov, axis=1),
        "OV_median": np.nanmedian(ov, axis=1),
        "OV_sd":     np.nanstd(ov, axis=1, ddof=0),
    }

# Plot helpers (each saves a PDF)
def _time_series_generic(date, y, title, ylabel, save_name=None):
    fig, ax = plt.subplots(figsize=(11, 5))
    ax.plot(date, y, marker="o", linewidth=1.5, label=ylabel)
    ax.set_xlim(date.min() - pd.Timedelta(days=2), date.max() + pd.Timedelta(days=2))
    ax.margins(x=0.0, y=0.08)
    ax.set_title(title)
    ax.set_xlabel("Date")
    ax.set_ylabel(ylabel)
    ax.xaxis.set_major_locator(mdates.MonthLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter("%b"))
    ax.grid(True, alpha=0.3)
    ax.legend(loc="best")
    plt.tight_layout()
    _savefig_pdf(save_name or f"{ylabel}__time_series")

def _time_series_target(date, y, title, ylabel, save_name=None):
    """Time series for TARGET: ALWAYS highlight 0s in red."""
    y = np.asarray(y, dtype=float)
    zero_mask = (y == 0)
    fig, ax = plt.subplots(figsize=(11, 5))
    ax.plot(date, y, marker="o", linewidth=1.5, label=ylabel)
    if np.any(zero_mask):
        ax.scatter(date[zero_mask], y[zero_mask], s=90, color="red", zorder=5, label="Zero (treated as missing)")
    ax.set_xlim(date.min() - pd.Timedelta(days=2), date.max() + pd.Timedelta(days=2))
    ax.margins(x=0.0, y=0.08)
    ax.set_title(title)
    ax.set_xlabel("Date")
    ax.set_ylabel(ylabel)
    ax.xaxis.set_major_locator(mdates.MonthLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter("%b"))
    ax.grid(True, alpha=0.3)
    ax.legend(loc="best")
    plt.tight_layout()
    _savefig_pdf(save_name or f"{ylabel}__time_series_target")

def _ribbon(date, y_min, y_max, y_med, title, ylabel, range_label="Range (min–max)", med_label="Median", save_name=None):
    fig, ax = plt.subplots(figsize=(11, 5))
    ax.fill_between(date, y_min, y_max, alpha=0.2, label=range_label)
    ax.plot(date, y_med, marker="o", linewidth=1.5, label=med_label)
    ax.set_xlim(date.min() - pd.Timedelta(days=2), date.max() + pd.Timedelta(days=2))
    ax.margins(x=0.0, y=0.08)
    ax.set_title(title)
    ax.set_xlabel("Date")
    ax.set_ylabel(ylabel)
    ax.xaxis.set_major_locator(mdates.MonthLocator())
    ax.xaxis.set_major_formatter(mdates.DateFormatter("%b"))
    ax.grid(True, alpha=0.3)
    ax.legend(loc="best")
    plt.tight_layout()
    _savefig_pdf(save_name or f"{ylabel}__ribbon")

def _hist_generic(y, title, xlabel, save_name=None):
    fig, ax = plt.subplots(figsize=(7, 4))
    ax.hist(pd.Series(y).dropna(), bins="auto")
    ax.set_title(title)
    ax.set_xlabel(xlabel)
    ax.set_ylabel("Frequency")
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    _savefig_pdf(save_name or f"{xlabel}__hist")

def _hist_target(y_all, title, xlabel, save_name=None):
    y_all = pd.Series(y_all, dtype=float)
    total_count   = int(y_all.notna().sum())
    zero_count    = int((y_all == 0).sum())
    nonzero_vals  = y_all[(y_all != 0) & y_all.notna()]
    nonzero_count = int(nonzero_vals.shape[0])

    fig, ax = plt.subplots(figsize=(7, 4))
    counts, bins, _ = ax.hist(nonzero_vals.to_numpy(), bins="auto", label=f"Non-zero: {nonzero_count}")

    if zero_count > 0:
        width = (bins[1] - bins[0]) * 0.9 if len(bins) >= 2 else 1.0
        ax.bar(0, zero_count, width=width, color="red", alpha=0.6,
               label=f"Zeros: {zero_count} | Total: {total_count}", zorder=5)
    else:
        ax.bar(0, 0, width=1.0, alpha=0.0, label=f"Zeros: 0 | Total: {total_count}")

    ax.set_title(title)
    ax.set_xlabel(xlabel)
    ax.set_ylabel("Frequency")
    ax.grid(True, alpha=0.3)
    ax.legend(loc="best")
    plt.tight_layout()
    _savefig_pdf(save_name or f"{xlabel}__hist_target")

def _scatter_env_vs_target(x, y_target, title, xlabel, ylabel, save_name=None):
    """
    Scatter of ENV feature vs TARGET with 0s highlighted in red.
    """
    x = np.asarray(x, dtype=float)
    y = np.asarray(y_target, dtype=float)
    zero_mask = (y == 0)
    ok = ~np.isnan(x) & ~np.isnan(y)

    fig, ax = plt.subplots(figsize=(6.8, 6.8))
    ax.scatter(x[ok & ~zero_mask], y[ok & ~zero_mask], marker="o", alpha=0.9, label="Observed")
    if np.any(ok & zero_mask):
        ax.scatter(x[ok & zero_mask], y[ok & zero_mask], s=90, color="red", zorder=5, label="Target == 0 (missing)")

    if np.any(ok):
        x_min, x_max = np.nanmin(x[ok]), np.nanmax(x[ok])
        y_min, y_max = np.nanmin(y[ok]), np.nanmax(y[ok])
        span_x = x_max - x_min if np.isfinite(x_max - x_min) else 1.0
        span_y = y_max - y_min if np.isfinite(y_max - y_min) else 1.0
        pad_x = max(0.5, 0.06 * span_x)
        pad_y = max(0.5, 0.06 * span_y)
        ax.set_xlim(x_min - pad_x, x_max + pad_x)
        ax.set_ylim(y_min - pad_y, y_max + pad_y)

    ax.set_title(title)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.grid(True, alpha=0.3)
    ax.legend(loc="best")
    plt.tight_layout()
    _savefig_pdf(save_name or f"scatter__{_slug(ylabel)}__vs__{_slug(xlabel)}")

def _corr_heatmap(df_num: pd.DataFrame, title: str, save_name=None):
    corr = df_num.corr(method="pearson")
    fig, ax = plt.subplots(figsize=(10, 8))
    im = ax.imshow(corr, aspect="auto", interpolation="nearest")
    ax.set_title(title)
    ax.set_xticks(range(corr.shape[1]))
    ax.set_yticks(range(corr.shape[0]))
    ax.set_xticklabels(corr.columns, rotation=45, ha="right")
    ax.set_yticklabels(corr.index)
    cbar = plt.colorbar(im, ax=ax)
    cbar.set_label("Pearson r")
    plt.tight_layout()
    _savefig_pdf(save_name or "correlation_heatmap")

# PIPELINE
# Load, tidy, window
df0 = pd.read_csv(CSV_PATH, sep=None, engine="python")
df0 = _standardize_columns(df0)
df0 = _parse_and_sort_dates(df0, DATE_COL)

# ENV features (FULL day + OVERNIGHT), computed BEFORE windowing (to keep prev-day context)
agg_cols = []
for base in ENV_VARS:
    mat24 = _env_matrix_from_vector_column(df0, base)  # (n,24), floats
    # full-day
    full_stats = _full_day_stats(mat24)
    for k, v in full_stats.items():
        col = f"{base}_{k}"
        df0[col] = v
        agg_cols.append(col)
    # overnight
    ov_stats = _overnight_stats(mat24)
    for k, v in ov_stats.items():
        col = f"{base}_{k}"
        df0[col] = v
        agg_cols.append(col)

# Targets (mark zeros and non-zero convenience)
for tgt in TARGETS:
    if tgt not in df0.columns:
        raise ValueError(f"TARGET '{tgt}' not found in CSV.")
    df0[f"{tgt}_is_zero"] = (df0[tgt] == 0)

# Filter window for plotting
df = _filter_window(df0, DATE_COL, WINDOW_START, WINDOW_END)
date = df[DATE_COL].to_numpy()

# PLOTS
# Precipitation (generic time series + hist)
if PRECIP_COL in df.columns:
    _time_series_generic(date, df[PRECIP_COL].to_numpy(), f"{PRECIP_COL} - Daily", PRECIP_COL,
                         save_name="precipitation__time_series")
    _hist_generic(df[PRECIP_COL], f"{PRECIP_COL} - Distribution", PRECIP_COL,
                  save_name="precipitation__hist")
else:
    print(f"[WARN] '{PRECIP_COL}' not found; skipping precip plots.")

# ENV ribbons & hist - FULL day
for base in ENV_VARS:
    _ribbon(date,
            df[f"{base}_FULL_min"].to_numpy(),
            df[f"{base}_FULL_max"].to_numpy(),
            df[f"{base}_FULL_median"].to_numpy(),
            title=f"{base} - FULL DAY Range (min–max) with Median",
            ylabel=f"{base} (full day)",
            range_label="Full-day range (min–max)",
            med_label="Full-day median",
            save_name=f"{base}__FULL__ribbon")
    _hist_generic(df[f"{base}_FULL_median"], f"{base} - FULL Median Distribution",
                  f"{base} (FULL median)", save_name=f"{base}__FULL__median_hist")
    _hist_generic(df[f"{base}_FULL_sd"],     f"{base} - FULL SD Distribution",
                  f"{base} (FULL sd)", save_name=f"{base}__FULL__sd_hist")

# ENV ribbons & hist - OVERNIGHT
for base in ENV_VARS:
    _ribbon(date,
            df[f"{base}_OV_min"].to_numpy(),
            df[f"{base}_OV_max"].to_numpy(),
            df[f"{base}_OV_median"].to_numpy(),
            title=f"{base} - OVERNIGHT Range (min–max) with Median",
            ylabel=f"{base} (overnight)",
            range_label="Overnight range (min–max)",
            med_label="Overnight median",
            save_name=f"{base}__OVERNIGHT__ribbon")
    _hist_generic(df[f"{base}_OV_median"], f"{base} - OVERNIGHT Median Distribution",
                  f"{base} (OV median)", save_name=f"{base}__OVERNIGHT__median_hist")
    _hist_generic(df[f"{base}_OV_sd"],     f"{base} - OVERNIGHT SD Distribution",
                  f"{base} (OV sd)", save_name=f"{base}__OVERNIGHT__sd_hist")

# Targets: ALWAYS highlight zeros
for tgt in TARGETS:
    y_all = df[tgt].to_numpy(dtype=float)
    _time_series_target(date, y_all, f"{tgt} - Daily Counts (Zeros highlighted)", tgt,
                        save_name=f"{tgt}__time_series")
    _hist_target(y_all, f"{tgt} - Distribution (Non-zero + Zero bar)", tgt,
                 save_name=f"{tgt}__hist")

# Scatter: target vs ENV medians (FULL and OVERNIGHT), with zeros highlighted
for tgt in TARGETS:
    y_all = df[tgt].to_numpy(dtype=float)
    for base in ENV_VARS:
        x_full = df[f"{base}_FULL_median"].to_numpy()
        _scatter_env_vs_target(x_full, y_all,
                               f"{tgt} vs {base} (FULL median) - Daily",
                               f"{base} (FULL median)", tgt,
                               save_name=f"scatter__{tgt}__{base}__FULL_median")
        x_ov = df[f"{base}_OV_median"].to_numpy()
        _scatter_env_vs_target(x_ov, y_all,
                               f"{tgt} vs {base} (OVERNIGHT median) - Daily",
                               f"{base} (OV median)", tgt,
                               save_name=f"scatter__{tgt}__{base}__OV_median")

# Correlation heatmap (using targets as values but treating zeros as NaN)
corr_df = pd.DataFrame(index=df.index)
for base in ENV_VARS:
    for stat in ["FULL_min", "FULL_max", "FULL_median", "FULL_sd", "OV_min", "OV_max", "OV_median", "OV_sd"]:
        corr_df[f"{base}_{stat}"] = df[f"{base}_{stat}"]
if PRECIP_COL in df.columns:
    corr_df[PRECIP_COL] = df[PRECIP_COL]
for tgt in TARGETS:
    corr_df[tgt] = df[tgt].where(df[tgt] != 0, np.nan)  # zeros -> NaN for correlation
corr_df = corr_df.apply(pd.to_numeric, errors="coerce")
_corr_heatmap_name = "correlation__FULL_OV_env_precip_targets"
_corr_heatmap(corr_df, "Correlation - FULL & OVERNIGHT ENV features, Precipitation, Targets (zeros→NaN)",
              save_name=_corr_heatmap_name)

# Save aggregated (optional)
if SAVE_AGGREGATED_CSV:
    keep = [DATE_COL] + ([PRECIP_COL] if PRECIP_COL in df0.columns else []) + agg_cols + TARGETS
    df0[keep].to_csv(AGG_OUT_PATH, index=False)
    print(f"[saved] {AGG_OUT_PATH}")


/var/folders/ck/yp00zrfx3pz5x6_mjzfr6hy00000gn/T/ipykernel_71199/539248373.py:115: RuntimeWarning: All-NaN slice encountered
  "FULL_min":    np.nanmin(mat24, axis=1),
/var/folders/ck/yp00zrfx3pz5x6_mjzfr6hy00000gn/T/ipykernel_71199/539248373.py:116: RuntimeWarning: All-NaN slice encountered
  "FULL_max":    np.nanmax(mat24, axis=1),
/Users/pradumchauhan/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1217: RuntimeWarning: All-NaN slice encountered
  r, k = function_base._ureduce(a, func=_nanmedian, axis=axis, out=out,
/Users/pradumchauhan/opt/anaconda3/lib/python3.9/site-packages/numpy/lib/nanfunctions.py:1878: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/var/folders/ck/yp00zrfx3pz5x6_mjzfr6hy00000gn/T/ipykernel_71199/539248373.py:133: RuntimeWarning: All-NaN slice encountered
  "OV_min":    np.nanmin(ov, axis=1),
/var/folders/ck/yp00zrfx3pz5x6_mjzfr6hy00000gn/T/ipykernel_71199/539248373.py:134: R

[saved] precipitation__time_series.pdf
[saved] precipitation__hist.pdf
[saved] Temperature__FULL__ribbon.pdf
[saved] Temperature__FULL__median_hist.pdf
[saved] Temperature__FULL__sd_hist.pdf
[saved] Humidity__FULL__ribbon.pdf
[saved] Humidity__FULL__median_hist.pdf
[saved] Humidity__FULL__sd_hist.pdf
[saved] Dew_Point__FULL__ribbon.pdf
[saved] Dew_Point__FULL__median_hist.pdf
[saved] Dew_Point__FULL__sd_hist.pdf
[saved] Water_Temperature__FULL__ribbon.pdf
[saved] Water_Temperature__FULL__median_hist.pdf
[saved] Water_Temperature__FULL__sd_hist.pdf
[saved] Temperature__OVERNIGHT__ribbon.pdf
[saved] Temperature__OVERNIGHT__median_hist.pdf
[saved] Temperature__OVERNIGHT__sd_hist.pdf
[saved] Humidity__OVERNIGHT__ribbon.pdf
[saved] Humidity__OVERNIGHT__median_hist.pdf
[saved] Humidity__OVERNIGHT__sd_hist.pdf
[saved] Dew_Point__OVERNIGHT__ribbon.pdf
[saved] Dew_Point__OVERNIGHT__median_hist.pdf
[saved] Dew_Point__OVERNIGHT__sd_hist.pdf
[saved] Water_Temperature__OVERNIGHT__ribbon.pdf
[saved]